In [1]:
import pandas as pd
import pprint
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import math
from pylab import rcParams
rcParams['figure.figsize'] = 25, 13

In [2]:
from pymongo import MongoClient

from IPython.core.display import display, HTML
[display(HTML("<style>.container { width:100% !important; }</style>"))];

In [8]:
with MongoClient('mongodb://localhost:27017/') as client:
    database = client["nyc_crashes"]
    crashes = database.crashes
    crashes_by_year = database.crashes_by_year

In [87]:
def out(Year):
    data = crashes_by_year.find_one({"_id": Year})['by_month']
    
    x_values = list(crashes_by_year.find_one({"_id": Year})['by_month'].keys())
    y_values_injured = []
    y_values_killed = []
    for month in data:
        injured = 0
        killed = 0
        injured = data[month]['pedestrians']['injured'] + data[month]['cyclists']['injured'] + data[month]['motorists']['injured']
        killed = data[month]['pedestrians']['killed'] + data[month]['cyclists']['killed'] + data[month]['motorists']['killed']
        y_values_injured += [injured]
        y_values_killed += [killed]
    
    show_graph(x_values, y_values_injured, y_values_killed)
    return

In [103]:
def show_graph(x_values, y_values_injured, y_values_killed):
    plt.figure(figsize=(10,5))

    plt.bar(x_values, y_values_killed, color=(0.62,0.07,0.04))
    plt.bar(x_values, y_values_injured, bottom=y_values_killed, color=(0.80,0.52,0.00))
    plt.show()

In [104]:
interact(out, Year=[data['_id'] for data in crashes_by_year.find()]);

interactive(children=(Dropdown(description='Year', options=('2020', '2019', '2013', '2016'), value='2020'), Ou…

In [114]:
def out3(Month):
    return Month

In [126]:
def out2(Year):
    #return interact(out3, Month=list(range(1, 13)));
    #return interact(out3, Month=(1,12));
    return interact(out3, Month=widgets.IntSlider(min=1,max=12,step=1,value=1))

In [127]:
interact(out2, Year=[data['_id'] for data in crashes_by_year.find()]);

interactive(children=(Dropdown(description='Year', options=('2020', '2019', '2013', '2016'), value='2020'), Ou…